In [1]:
#%%===== Initialization =====%%#
#===== Import Libs =====#
print('Importing libraries...')

import numpy as np
import pandas as pd
import json as js
import requests
import folium

from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.exc import GeocoderTimedOut


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt # plotting library
import matplotlib.colors as colors

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')
#===========================#
#===== Define Foursquare Credentials and other variables =====#
print('Defining Foursquare credentials...')
CLIENT_ID = 'BIJW5DEUCZQWIW4DIUQHYVJDRNBIFH0PHNPWUX1PFYRVTJJY' # your Foursquare ID
CLIENT_SECRET = 'NM1S4N0GGK2F2FDXRQFL50AR4K4SSY0USJWVOTMCCITL3DQ2' # your Foursquare Secret
TRULIA_KEY = "b9735tmvu3y3w9qs8hajydbe"
VERSION = '20180605' # Foursquare API version
search_city = 'Birmingham' # 'Tuscaloosa'
compare_city = 'Huntsville'# 'Northport'
search_state = 'AL'
compare_state = 'AL'
search_zip = '35201' # 35223 for Mountain Brook. '35401'
compare_zip = '35801' # '35473'
search_country = 'USA'
compare_country = 'USA'
searchOnly_City = search_city
searchPlace = search_city+','+search_state+','+search_zip+','+search_country # The name of the target metropolitan area.
searchPlace_compare = compare_city+','+compare_state+','+compare_zip+','+compare_country # The name of the comparison target metropolicatan area
searchEverywhere = False # Search the entire Birmingham metropolitan area. Caution: Very large, may exceed the Foursquare daily call limit!
if searchEverywhere == False:
    searchNPlaces = 10 # Only search the first 5 neighborhoods
compareMode = True # Compare the two cities
explore_radius = 1500 # The radius around a City to be explored
venue_limit = 10 # The first 30 venues near a City
num_clusters = 2 # Number of clusters in the K-Mean clustering method
map_zoom_scale = 15 # Scale of the map
print('Explore radius: {}, venue limit: {}'.format(explore_radius,venue_limit))
if searchEverywhere == False:
    print('Only search the venues in this City: {}'.format(searchOnly_City))
#=====================================================#
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Foursquare credentials defined successfully.')
print('Search all the neiborhoods in the {} metropolitan area: {}'.format(searchPlace, searchEverywhere))

if searchEverywhere == True:
    print('Warning: You are searching the venues in the entire {} metropolitan area. May exceed the Foursquare daily call limit. Proceed with caution!'.format(searchPlace))
    print("If 'KeyError: groups' appears, you reach the Foursquare daily call limit. Please search 24 hours later.")
#=====================================================#
##========================##

Importing libraries...
Libraries imported.
Defining Foursquare credentials...
Explore radius: 1500, venue limit: 10
Only search the venues in this City: Birmingham
Your credentails:
CLIENT_ID: BIJW5DEUCZQWIW4DIUQHYVJDRNBIFH0PHNPWUX1PFYRVTJJY
CLIENT_SECRET:NM1S4N0GGK2F2FDXRQFL50AR4K4SSY0USJWVOTMCCITL3DQ2
Foursquare credentials defined successfully.
Search all the neiborhoods in the Birmingham,AL,35201,USA metropolitan area: False


In [2]:
#%%===== Read the postal code table from the designated website =====%%#
print('Reading Alabama zip data...')

Url_AlaZip = 'https://www.zipcodestogo.com/Alabama/'
# Read the zip codes #
AlaZip_html = requests.get(Url_AlaZip).text

zipCodes_Soup = BeautifulSoup(AlaZip_html,'lxml')
print(zipCodes_Soup.prettify()) #For test use. "soup.prettify is a method that is used to display the html file in a readable way"

zipCodes_table = zipCodes_Soup.find('table', class_="inner_table")
print(zipCodes_table.prettify()) # For test use. Display the html in a readable way
# print(type(zipCodes_table)) # For test use.

zipCodes_list = [] # Create an empty array that can be used to store the zip codes
city_list = [] # Create an empty array that can be used to store the cities
county_list = [] # Create an empty array that can be used to store the counties
latitude_list = [] # Create an empty array that can be used to store the latitude
longitude_list = [] # Create an empty array that can be used to store the longitude


for row in zipCodes_table.findAll("tr"):
    cells = row.findAll("td")
    # For each "tr", assign each "td" to a variable.
    if len(cells) == 4: # Number of columns in the table
        zipCodes_txt = cells[0].find(text=True) # Index from the first element
        city_txt = cells[1].find(text=True)
        county_txt = cells[2].find(text=True)
        if zipCodes_txt != 'Zip Code' and city_txt != 'City' and county_txt != 'County':
            print('Post code:',zipCodes_txt,',', 'City:',city_txt,',', 'County:',county_txt) # For test use
        
            zipCodes_list.append(zipCodes_txt)
            city_list.append(city_txt)
            county_list.append(county_txt)

list_index = 0
for zip_code, city, county in zip(zipCodes_list, city_list, county_list):
    geolocator = Nominatim(user_agent="Week_4_Coursera_Data Science_Capstone Project")
#    City, State, Zip, Country
    temp_place = city+', '+county+', '+'AL'+', '+zip_code+', '+'USA'
    temp_location = geolocator.geocode(temp_place, timeout=None)
    if temp_location is not None and temp_location.latitude and temp_location.longitude is not None:
        latitude_txt = temp_location.latitude
        longitude_txt = temp_location.longitude
        latitude_list.insert(list_index, latitude_txt)
        longitude_list.insert(list_index, longitude_txt)
        print('Current place: {}, Latitude: {}, Longitude: {}, Count: {}'.format(temp_place, latitude_txt, longitude_txt, list_index))
    else:
        latitude_list.insert(list_index, 'NaN')
        longitude_list.insert(list_index, 'NaN')
        print('Current place: {}, Latitude: {}, Longitude: {}, Count: {}'.format(temp_place, latitude_txt, longitude_txt, list_index))
    list_index = list_index + 1

df_AlaZipCodes = pd.DataFrame()
df_AlaZipCodes['Postal Code'] = zipCodes_list
df_AlaZipCodes['City'] = city_list
df_AlaZipCodes['County'] = county_list
df_AlaZipCodes['Latitude'] = latitude_list
df_AlaZipCodes['Longitude'] = longitude_list

df_AlaZipCodes.dropna(axis=0, how='any', inplace=True)
df_AlaZipCodes.drop_duplicates(subset ="Postal Code", 
                               keep = 'first', inplace = True) 
df_AlaZipCodes.drop_duplicates(subset =['Latitude', 'Longitude'], 
                               keep = 'first', inplace = True)

df_AlaZipCodes.head()
print('Alabama Postal Code dataframe has been created successfully.')
##==========================##

Reading Alabama zip data...
<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   ZIP Code List for Alabama
  </title>
  <meta content="Find zip code by searching by city, state, or zip code" name="description"/>
  <meta content="city zip codes county state us usa zipcodes by state" name="keywords"/>
  <style type="text/css">
   body {
			font-size: 16px;
			font-family:Arial;
    	}
		.small {
			font-size: 12px; /* 75% of the baseline */	
		}
		.large {
			font-size: 20px; /* 125% of the baseline */
		}
		.bold {
			font-weight:bold;	
		}
		.inner_table {
			padding: 1px;
			border-spacing: 0px;
			width: 100%;
		}
		inner_table tr {
			border: 0px;
			padding: 0px;
		}
		.inner_table td {
			padding: 3px;
			border: grey solid 1px;
		}
		
		tr:nth-child(even) {
			background-color: #FFFFCC;
		}
		tr:nth-child(odd) {
			background-color: #FFFFFF;
		}
		#header img {
			display: none;
		}

Post code: 35207 , City: Birmingham , County: Jefferson
Post code: 35208 , City: Birmingham , County: Jefferson
Post code: 35209 , City: Birmingham , County: Jefferson
Post code: 35210 , City: Birmingham , County: Jefferson
Post code: 35211 , City: Birmingham , County: Jefferson
Post code: 35212 , City: Birmingham , County: Jefferson
Post code: 35213 , City: Birmingham , County: Jefferson
Post code: 35214 , City: Birmingham , County: Jefferson
Post code: 35215 , City: Birmingham , County: Jefferson
Post code: 35216 , City: Birmingham , County: Jefferson
Post code: 35217 , City: Birmingham , County: Jefferson
Post code: 35218 , City: Birmingham , County: Jefferson
Post code: 35219 , City: Birmingham , County: Jefferson
Post code: 35220 , City: Birmingham , County: Jefferson
Post code: 35221 , City: Birmingham , County: Jefferson
Post code: 35222 , City: Birmingham , County: Jefferson
Post code: 35223 , City: Birmingham , County: Jefferson
Post code: 35224 , City: Birmingham , County: Je

Post code: 36022 , City: Deatsville , County: Elmore
Post code: 36023 , City: East Tallassee , County: Tallapoosa
Post code: 36024 , City: Eclectic , County: Elmore
Post code: 36025 , City: Elmore , County: Elmore
Post code: 36026 , City: Equality , County: Coosa
Post code: 36027 , City: Eufaula , County: Barbour
Post code: 36028 , City: Dozier , County: Crenshaw
Post code: 36029 , City: Fitzpatrick , County: Bullock
Post code: 36030 , City: Forest Home , County: Butler
Post code: 36031 , City: Fort Davis , County: Macon
Post code: 36032 , City: Fort Deposit , County: Lowndes
Post code: 36033 , City: Georgiana , County: Butler
Post code: 36034 , City: Glenwood , County: Crenshaw
Post code: 36035 , City: Goshen , County: Pike
Post code: 36036 , City: Grady , County: Montgomery
Post code: 36037 , City: Greenville , County: Butler
Post code: 36038 , City: Gantt , County: Covington
Post code: 36039 , City: Hardaway , County: Macon
Post code: 36040 , City: Hayneville , County: Lowndes
Post 

Post code: 36457 , City: Megargel , County: Monroe
Post code: 36458 , City: Mexia , County: Monroe
Post code: 36460 , City: Monroeville , County: Monroe
Post code: 36461 , City: Monroeville , County: Monroe
Post code: 36462 , City: Monroeville , County: Monroe
Post code: 36467 , City: Opp , County: Covington
Post code: 36470 , City: Perdue Hill , County: Monroe
Post code: 36471 , City: Peterman , County: Monroe
Post code: 36473 , City: Range , County: Conecuh
Post code: 36474 , City: Red Level , County: Covington
Post code: 36475 , City: Repton , County: Conecuh
Post code: 36476 , City: River Falls , County: Covington
Post code: 36477 , City: Samson , County: Geneva
Post code: 36480 , City: Uriah , County: Monroe
Post code: 36481 , City: Vredenburgh , County: Monroe
Post code: 36482 , City: Whatley , County: Clarke
Post code: 36483 , City: Wing , County: Covington
Post code: 36501 , City: Alma , County: Clarke
Post code: 36502 , City: Atmore , County: Escambia
Post code: 36503 , City: 

Current place: Moody, Saint Clair, AL, 35004, USA, Latitude: 33.5909365, Longitude: -86.4908168, Count: 0
Current place: Adamsville, Jefferson, AL, 35005, USA, Latitude: 33.6009389, Longitude: -86.9561059, Count: 1
Current place: Adger, Jefferson, AL, 35006, USA, Latitude: 33.3770561, Longitude: -87.0936065, Count: 2
Current place: Alabaster, Shelby, AL, 35007, USA, Latitude: 33.2442813, Longitude: -86.8163773, Count: 3
Current place: Alexander City, Tallapoosa, AL, 35010, USA, Latitude: 32.944012, Longitude: -85.9538532, Count: 4
Current place: Alexander City, Tallapoosa, AL, 35011, USA, Latitude: 32.944012, Longitude: -85.9538532, Count: 5
Current place: Allgood, Blount, AL, 35013, USA, Latitude: 33.9120429, Longitude: -86.5094315, Count: 6
Current place: Alpine, Talladega, AL, 35014, USA, Latitude: 33.348722, Longitude: -86.2385859, Count: 7
Current place: Alton, Jefferson, AL, 35015, USA, Latitude: 33.5792704, Longitude: -86.637487, Count: 8
Current place: Arab, Marshall, AL, 35016

...

Current place: Lapine, Montgomery, AL, 36046, USA, Latitude: 31.9657055, Longitude: -86.2844093, Count: 456
Current place: Letohatchee, Lowndes, AL, 36047, USA, Latitude: 32.129589, Longitude: -86.4858058, Count: 457
Current place: Louisville, Barbour, AL, 36048, USA, Latitude: 31.7834963, Longitude: -85.5557751, Count: 458
Current place: Luverne, Crenshaw, AL, 36049, USA, Latitude: 31.7165455, Longitude: -86.2638492, Count: 459
Current place: Marbury, Autauga, AL, 36051, USA, Latitude: 32.7012405, Longitude: -86.4710876, Count: 460
Current place: Mathews, Montgomery, AL, 36052, USA, Latitude: 32.2654206, Longitude: -86.0038464, Count: 461
Current place: Midway, Bullock, AL, 36053, USA, Latitude: 32.077098, Longitude: -85.522167, Count: 462
Current place: Millbrook, Elmore, AL, 36054, USA, Latitude: 32.4798575, Longitude: -86.3619163, Count: 463
Current place: Mount Meigs, Montgomery, AL, 36057, USA, Latitude: 32.3626393, Longitude: -86.1019068, Count: 464
Current place: Perote, Bulloc

Current place: Alexandria, Calhoun, AL, 36250, USA, Latitude: 33.7739892, Longitude: -85.885523, Count: 531
Current place: Ashland, Clay, AL, 36251, USA, Latitude: 33.2737256, Longitude: -85.8360741, Count: 532
Current place: Bynum, Calhoun, AL, 36253, USA, Latitude: 33.6131592, Longitude: -85.9610793, Count: 533
Current place: Choccolocco, Calhoun, AL, 36254, USA, Latitude: 33.6592714, Longitude: -85.7035728, Count: 534
Current place: Cragford, Clay, AL, 36255, USA, Latitude: 33.2509719, Longitude: -85.6723857, Count: 535
Current place: Daviston, Tallapoosa, AL, 36256, USA, Latitude: 33.0528992, Longitude: -85.6396783, Count: 536
Current place: De Armanville, Calhoun, AL, 36257, USA, Latitude: 33.0528992, Longitude: -85.6396783, Count: 537
Current place: Delta, Clay, AL, 36258, USA, Latitude: 33.4401108, Longitude: -85.690515, Count: 538
Current place: Eastaboga, Calhoun, AL, 36260, USA, Latitude: 33.6059367, Longitude: -86.0213587, Count: 539
Current place: Edwardsville, Cleburne, AL

Current place: Excel, Monroe, AL, 36439, USA, Latitude: 31.4279433, Longitude: -87.3413731, Count: 608
Current place: Flomaton, Escambia, AL, 36441, USA, Latitude: 31.000182, Longitude: -87.2608071, Count: 609
Current place: Florala, Covington, AL, 36442, USA, Latitude: 31.00518, Longitude: -86.3280018, Count: 610
Current place: Franklin, Monroe, AL, 36444, USA, Latitude: 31.7145996, Longitude: -87.4113791, Count: 611
Current place: Frisco City, Monroe, AL, 36445, USA, Latitude: 31.4334985, Longitude: -87.401375, Count: 612
Current place: Fulton, Clarke, AL, 36446, USA, Latitude: 31.7884881, Longitude: -87.7272221, Count: 613
Current place: Goodway, Monroe, AL, 36449, USA, Latitude: 31.3368355, Longitude: -87.4258186, Count: 614
Current place: Grove Hill, Clarke, AL, 36451, USA, Latitude: 31.7087681, Longitude: -87.7772218, Count: 615
Current place: Kinston, Coffee, AL, 36453, USA, Latitude: 31.2160045, Longitude: -86.1710579, Count: 616
Current place: Lenox, Conecuh, AL, 36454, USA, L

Current place: Robertsdale, Baldwin, AL, 36567, USA, Latitude: 30.5538048, Longitude: -87.7119324, Count: 685
Current place: Saint Elmo, Mobile, AL, 36568, USA, Latitude: 30.5035291, Longitude: -88.2541711, Count: 686
Current place: Saint Stephens, Washington, AL, 36569, USA, Latitude: 31.5404411, Longitude: -88.0552823, Count: 687
Current place: Saraland, Mobile, AL, 36571, USA, Latitude: 30.820742, Longitude: -88.0705556, Count: 688
Current place: Satsuma, Mobile, AL, 36572, USA, Latitude: 30.853241, Longitude: -88.0561106, Count: 689
Current place: Seminole, Baldwin, AL, 36574, USA, Latitude: 30.5151954, Longitude: -87.4738692, Count: 690
Current place: Semmes, Mobile, AL, 36575, USA, Latitude: 30.768046, Longitude: -88.257362, Count: 691
Current place: Silverhill, Baldwin, AL, 36576, USA, Latitude: 30.5454715, Longitude: -87.7516559, Count: 692
Current place: Spanish Fort, Baldwin, AL, 36577, USA, Latitude: 30.6749127, Longitude: -87.9152724, Count: 693
Current place: Stapleton, Ba

Current place: Jones, Autauga, AL, 36749, USA, Latitude: 32.5840214, Longitude: -86.8974878, Count: 763
Current place: Maplesville, Chilton, AL, 36750, USA, Latitude: 32.7884608, Longitude: -86.8716549, Count: 764
Current place: Lower Peach Tree, Wilcox, AL, 36751, USA, Latitude: 31.8407077, Longitude: -87.5452737, Count: 765
Current place: Lowndesboro, Lowndes, AL, 36752, USA, Latitude: 32.2751399, Longitude: -86.6113658, Count: 766
Current place: Mc Williams, Wilcox, AL, 36753, USA, Latitude: 32.2751399, Longitude: -86.6113658, Count: 767
Current place: Magnolia, Marengo, AL, 36754, USA, Latitude: 32.1357021, Longitude: -87.6680606, Count: 768
Current place: Marion, Perry, AL, 36756, USA, Latitude: 32.6323536, Longitude: -87.3191655, Count: 769
Current place: Plantersville, Dallas, AL, 36758, USA, Latitude: 32.6567975, Longitude: -86.9244334, Count: 770
Current place: Marion Junction, Dallas, AL, 36759, USA, Latitude: 32.437358, Longitude: -87.2388839, Count: 771
Current place: Minte

In [3]:
#%%===== Get the geo location information of the target metropolitan area by using Geocoder =====%%#
##===== Create a map of Birmingham =====##
geolocator = Nominatim(user_agent="Week_4_Coursera_Data Science_Capstone Project")
metro_location = geolocator.geocode(searchPlace,timeout=None)
print(metro_location)
metro_latitude = metro_location.latitude
metro_longitude = metro_location.longitude
print('The geograpical coordinate of {} is {}, {}.'.format(searchPlace, metro_latitude, metro_longitude))
# create map of the target metropolitan area using latitude and longitude values
map_metro = folium.Map(location=[metro_latitude, metro_longitude], zoom_start=map_zoom_scale)

label = '{}, {}'.format(search_city, search_state)
label = folium.Popup(label)
folium.CircleMarker(
    [metro_latitude, metro_longitude],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,).add_to(map_metro)  
map_metro
# print('Saving the map...')
# map_save_name = map_save_path + search_city + '.html'
# map_metro.save(map_save_name)
# print('{} map saved successfully.'.format(search_city))
##===========================================================##
##===== Create a map Huntsville =====##
geolocator = Nominatim(user_agent="Week_4_Coursera_Data Science_Capstone Project")
compare_location = geolocator.geocode(searchPlace_compare,timeout=None)
print(compare_location)
compare_latitude = compare_location.latitude
compare_longitude = compare_location.longitude
print('The geograpical coordinate of {} is {}, {}.'.format(compare_location, compare_latitude, compare_longitude))
# create map of the target metropolitan area using latitude and longitude values
map_compare = folium.Map(location=[compare_latitude, compare_longitude], zoom_start=map_zoom_scale)

label = '{}, {}'.format(compare_city, compare_state)
label = folium.Popup(label)
folium.CircleMarker(
    [compare_latitude, compare_longitude],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,).add_to(map_compare) 
map_compare
# print('Saving the map...')
# map_save_name = map_save_path + compare_city + '.html'
# map_compare.save(map_save_name)
# print('{} map saved successfully.'.format(compare_city))
##====================================================================##

Birmingham, Jefferson County, Alabama, USA
The geograpical coordinate of Birmingham,AL,35201,USA is 33.5206824, -86.8024326.
Huntsville, Madison County, Alabama, USA
The geograpical coordinate of Huntsville, Madison County, Alabama, USA is 34.729847, -86.5859011.


In [4]:
#%%===== Define a function to explore the Cities =====%%#
# Try to modify this part to make it applicable to distinguish place by zip code## Zip Code line has a bug. Feed to many zip codes
def getNearbyVenues(zip_code, names, latitudes, longitudes, radius, LIMIT):
#                                   names = df_targetCity['City'], 
#                                   latitudes = df_targetCity['Latitude'], 
#                                   longitudes = df_targetCity['Longitude'],
#                                   radius = explore_radius,
#                                   LIMIT = venue_limit
    venues_list=[]
    for postal_code, name, lat, lng in zip(zip_code, names, latitudes, longitudes):
        print(postal_code, name) 
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
#        print(results) # For test only
        # return only relevant information for each nearby venue
        if len(results) == 0:   
            venues_list.append([(postal_code, 
                                name, 
                                lat, 
                                lng,
                                'NaN') for empty_count in range(LIMIT)])
        else:
            venues_list.append([(
            postal_code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    # for venue_list in venues_list:
        # for item in venue_list:
            # nearby_venues = pd.DataFrame(item)

    nearby_venues.columns = ['Postal Code',
                              'City', 
                              'City Latitude', 
                              'City Longitude', 
                              'Venue', 
                              'Venue Latitude', 
                              'Venue Longitude', 
                              'Venue Category']
    
    return(nearby_venues)
##=========================================================##

In [5]:
#%%===== Use the defined function to explore the Cities in Birmingham =====%%#
if searchEverywhere == False:    
#    df_targetCity = df_AlaZipCodes[df_AlaZipCodes.City == search_city] 
    df_temp = df_AlaZipCodes[df_AlaZipCodes.City == search_city]
    df_targetCity = df_temp.iloc[0:searchNPlaces,:]
else:
#    df_targetCity = df_temp
    df_targetCity = df_AlaZipCodes
#df_targetCity.drop_duplicates(subset ="Postal Code", 
#                              keep = 'first', inplace = True) 
#df_targetCity.drop_duplicates(subset =['Latitude', 'Longitude'], 
#                              keep = 'first', inplace = True) 
df_targetCity.dropna(axis=0, inplace=True)
print(df_targetCity.head())

TargetCity_venues = getNearbyVenues(
                                   zip_code = df_targetCity['Postal Code'],
                                   names = df_targetCity['City'], 
                                   latitudes = df_targetCity['Latitude'], 
                                   longitudes = df_targetCity['Longitude'],
                                   radius = explore_radius,
                                   LIMIT = venue_limit
                                   )
TargetCity_venues.head()
##=============================================================================##

    Postal Code        City     County Latitude Longitude
114       35201  Birmingham  Jefferson  33.5207  -86.8024
152       35242  Birmingham     Shelby  33.4164  -86.6903
35201 Birmingham
35242 Birmingham


,Postal Code,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,35201,Birmingham,33.520682,-86.802433,Urban Standard,33.517925,-86.801356,Coffee Shop
1,35201,Birmingham,33.520682,-86.802433,El Barrio,33.516913,-86.803053,Mexican Restaurant
2,35201,Birmingham,33.520682,-86.802433,Bamboo On 2nd,33.517032,-86.803064,Asian Restaurant
3,35201,Birmingham,33.520682,-86.802433,Rogue Tavern,33.517813,-86.801667,American Restaurant
4,35201,Birmingham,33.520682,-86.802433,The Collins Bar,33.516507,-86.803835,Bar


In [6]:
#%%===== Use the defined function to explore the Cities in Huntsville as a comparison =====%%#
if compareMode == True:
#    df_compareCity = df_AlaZipCodes[df_AlaZipCodes.City == compare_city] 
    df_temp = df_AlaZipCodes[df_AlaZipCodes.City == compare_city]
    if searchEverywhere == False:
        df_compareCity = df_temp.iloc[0:searchNPlaces,:]
    else:
        df_compareCity = df_temp
else:
    df_compareCity = df_AlaZipCodes
#df_compareCity.drop_duplicates(subset = "Postal Code", 
#                               keep = 'first', inplace = True)
#df_compareCity.drop_duplicates(subset = ['Latitude', 'Longitude'], 
#                               keep = 'first', inplace = True)
df_compareCity.dropna(axis=0, inplace=True)
df_compareCity.head()


compareCity_venues = getNearbyVenues(
                                    zip_code = df_compareCity['Postal Code'],
                                    names = df_compareCity['City'], 
                                    latitudes = df_compareCity['Latitude'], 
                                    longitudes = df_compareCity['Longitude'],
                                    radius = explore_radius,
                                    LIMIT = venue_limit
                                    )
compareCity_venues.head()
##====================================================================##

35801 Huntsville
35806 Huntsville
35807 Huntsville


,Postal Code,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,35801,Huntsville,34.729847,-86.585901,"Sam And Greg's Pizzeria, Gelateria",34.730889,-86.585282,Pizza Place
1,35801,Huntsville,34.729847,-86.585901,Pints and Pixels,34.731771,-86.586118,Arcade
2,35801,Huntsville,34.729847,-86.585901,The Cozy Cow,34.728900,-86.587923,Coffee Shop
3,35801,Huntsville,34.729847,-86.585901,Big Spring Park,34.727492,-86.587273,Park
4,35801,Huntsville,34.729847,-86.585901,Commerce Kitchen,34.730139,-86.584412,American Restaurant


In [7]:
#%%===== Display the venue counts in each city =====%%#
print('There are {} unique venue categories in {}.'.format(len(TargetCity_venues['Venue Category'].unique()), search_city))
print('There are {} unique venue categories in {}.'.format(len(compareCity_venues['Venue Category'].unique()), compare_city))
##=====================================##

There are 19 unique venue categories in Birmingham.
There are 18 unique venue categories in Huntsville.


In [9]:
#%%===== Analyze each City =====%%#
##===== Birmingham first =====##
# one hot encoding
TargetCity_onehot = pd.get_dummies(TargetCity_venues[['Venue Category']], prefix="", prefix_sep="") # Convert categorical variable into dummy/indicator variables (True/False)
TargetCity_onehot.head()
# add City & Postal Code columns back to dataframe
TargetCity_onehot['Postal Code'] = TargetCity_venues['Postal Code']
TargetCity_onehot['City'] = TargetCity_venues['City'] 

# move City & Postal Code columns to the first column
fixed_columns = [TargetCity_onehot.columns[-1]] + list(TargetCity_onehot.columns[:-1])
TargetCity_onehot = TargetCity_onehot[fixed_columns]

fixed_columns = [TargetCity_onehot.columns[-1]] + list(TargetCity_onehot.columns[:-1])
TargetCity_onehot = TargetCity_onehot[fixed_columns]

TargetCity_onehot.head()

TargetCity_grouped = TargetCity_onehot.groupby(['Postal Code','City']).mean().reset_index()
TargetCity_grouped.head()
#################################
num_top_venues = 5

## Bug
for hood in TargetCity_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = TargetCity_grouped[TargetCity_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:] # Changed
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
#=========================================#

----35201----
                             venue  freq
0              American Restaurant   0.1
1                              Gym   0.1
2  Southern / Soul Food Restaurant   0.1
3               Mexican Restaurant   0.1
4                            Hotel   0.1


----35242----
                    venue  freq
0     American Restaurant   0.1
1  Furniture / Home Store   0.1
2                Tea Room   0.1
3             Supermarket   0.1
4          Sandwich Place   0.1




In [10]:
#%%===== Then Huntsville =====%%#
# one hot encoding
compareCity_onehot = pd.get_dummies(compareCity_venues[['Venue Category']], prefix="", prefix_sep="") # Convert categorical variable into dummy/indicator variables (True/False)
compareCity_onehot.head()
# add City & Postal Code columns back to dataframe
compareCity_onehot['Postal Code'] = compareCity_venues['Postal Code']
compareCity_onehot['City'] = compareCity_venues['City'] 

# move City column to the first column
fixed_columns = [compareCity_onehot.columns[-1]] + list(compareCity_onehot.columns[:-1])
compareCity_onehot = compareCity_onehot[fixed_columns]

fixed_columns = [compareCity_onehot.columns[-1]] + list(compareCity_onehot.columns[:-1])
compareCity_onehot = compareCity_onehot[fixed_columns]

compareCity_onehot.head()

compareCity_grouped = compareCity_onehot.groupby(['Postal Code','City']).mean().reset_index()
compareCity_grouped.head()

for hood in compareCity_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = compareCity_grouped[compareCity_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:] # Changed
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
#=========================================#
##=========================================##

----35801----
                  venue  freq
0   American Restaurant   0.2
1  Gym / Fitness Center   0.1
2             Speakeasy   0.1
3           Pizza Place   0.1
4                  Park   0.1


----35806----
         venue  freq
0        Hotel   0.2
1  Pizza Place   0.1
2    BBQ Joint   0.1
3       Bakery   0.1
4   Steakhouse   0.1


----35807----
                   venue  freq
0    American Restaurant   0.0
1  Outdoors & Recreation   0.0
2             Steakhouse   0.0
3             Sports Bar   0.0
4              Speakeasy   0.0




In [11]:
#%%===== Define a function to sort the venues in descending order =====%%#
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
##=================================================================##

In [13]:
#%%===== Display the top 10 venues in each city =====%%#
##===== Birmingham first =====##
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code','City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # For 1st, 2nd, and 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
            

# create a new dataframe
TargetCity_venues_sorted = pd.DataFrame(columns=columns)
TargetCity_venues_sorted['Postal Code'] = TargetCity_grouped['Postal Code']
TargetCity_venues_sorted['City'] = TargetCity_grouped['City']

for ind in np.arange(TargetCity_grouped.shape[0]):
    if TargetCity_grouped.shape[1] - 2 >= num_top_venues: # Check whether the number of the most popular venue categories is larger or equal to the user defined top X venue categories.
        TargetCity_venues_sorted.iloc[ind, 2:] = return_most_common_venues(TargetCity_grouped.iloc[ind, :], num_top_venues)
    else: # If the condition cannot be fulfilled, only give the values (popular venues) to the corresponding columns, then fill other columns with NaN
        TargetCity_venues_sorted.iloc[ind, 2:(TargetCity_grouped.shape[1])] = return_most_common_venues(TargetCity_grouped.iloc[ind, :], num_top_venues)

print('Here are the most common venues in {}:'.format(search_city))
TargetCity_venues_sorted
#====================================================#
##===== Then Huntsville =====##
# create columns according to number of top venues
columns = ['Postal Code','City'] # Initialize this variable for the next city

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
compareCity_venues_sorted = pd.DataFrame(columns=columns)
compareCity_venues_sorted['Postal Code'] = compareCity_grouped['Postal Code']
compareCity_venues_sorted['City'] = compareCity_grouped['City']

for ind in np.arange(compareCity_grouped.shape[0]):
    if compareCity_grouped.shape[1] - 2 >= num_top_venues: # Check whether the number of the most popular venue categories is larger or equal to the user defined top X venue categories.
        compareCity_venues_sorted.iloc[ind, 2:] = return_most_common_venues(compareCity_grouped.iloc[ind, :], num_top_venues)           
    else:
        compareCity_venues_sorted.iloc[ind, 2:(compareCity_grouped.shape[1])] = return_most_common_venues(compareCity_grouped.iloc[ind, :], num_top_venues)
            

print('Here are the most common venues in {}:'.format(compare_city))
compareCity_venues_sorted
#====================================================#
##=========================================================##

Here are the most common venues in Birmingham:
Here are the most common venues in Huntsville:


,Postal Code,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,35801,Huntsville,American Restaurant,Park,Arcade,Beer Store,Coffee Shop,Gym / Fitness Center,Sushi Restaurant,Pizza Place,Speakeasy,Outdoors & Recreation
1,35806,Huntsville,Hotel,Sports Bar,Salon / Barbershop,Pizza Place,BBQ Joint,Outdoors & Recreation,Steakhouse,Bakery,Grocery Store,Beer Store
2,35807,Huntsville,Sushi Restaurant,Gym / Fitness Center,Arcade,BBQ Joint,Bakery,Beer Store,Coffee Shop,Grocery Store,Hotel,Steakhouse


In [14]:
#%%===== Cluster neighborhoods -- Using K-mean clustering =====%%#
## Currently editing this part
# set number of clusters
if num_clusters <= TargetCity_grouped.shape[0]:
    kclusters = num_clusters
else:
    kclusters = TargetCity_grouped.shape[0]

##==== Birmingham first =====##
TargetCity_grouped_clustering = TargetCity_grouped.drop(['Postal Code','City'], 1) # Drop the neighborhood name column

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TargetCity_grouped_clustering)

## check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

TargetCity_merged = df_targetCity

TargetCity_merged.drop_duplicates(subset=['City', 'Postal Code'], keep=False)

# add clustering labels
TargetCity_merged['Cluster Labels'] = kmeans.labels_

TargetCity_merged.set_index('Postal Code', inplace=True)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TargetCity_venues_sorted.drop('City', axis=1, inplace=True)
TargetCity_merged = TargetCity_merged.join(TargetCity_venues_sorted.set_index('Postal Code'), on='Postal Code') # Change to merge may work
#TargetCity_merged.drop(['City_x', 'City_y'], axis=1, inplace=True)
TargetCity_merged.reset_index(inplace=True)
TargetCity_merged.head() # check the last columns!
##=============================================#

,Postal Code,City,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,35201,Birmingham,Jefferson,33.5207,-86.8024,0,Wine Bar,Hotel,Antique Shop,Asian Restaurant,Bar,Coffee Shop,Gym,American Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant
1,35242,Birmingham,Shelby,33.4164,-86.6903,1,Health Food Store,Pet Store,Cajun / Creole Restaurant,Cupcake Shop,Furniture / Home Store,Tea Room,American Restaurant,Pizza Place,Sandwich Place,Supermarket


In [15]:
#%%===== Then Huntsville =====%%#
# set number of clusters
if num_clusters <= compareCity_grouped.shape[0]:
    kclusters = num_clusters
else:
    kclusters = compareCity_grouped.shape[0]

##==== Birmingham first =====##
compareCity_grouped_clustering = compareCity_grouped.drop(['Postal Code','City'], 1) # Drop the neighborhood name column

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(compareCity_grouped_clustering)

## check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

compareCity_merged = df_compareCity # df_compareCity does not have ['Cluster Labels'] column

compareCity_merged.drop_duplicates(subset=['City', 'Postal Code'], keep=False)

# add clustering labels
compareCity_merged['Cluster Labels'] = kmeans.labels_

compareCity_merged.set_index('Postal Code', inplace=True)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
compareCity_venues_sorted.drop('City', axis=1, inplace=True)
compareCity_merged = compareCity_merged.merge(compareCity_venues_sorted.set_index('Postal Code'), on='Postal Code') # Change to merge may work
#compareCity_merged.drop(['City_x', 'City_y'], axis=1, inplace=True)
compareCity_merged.reset_index(inplace=True)
compareCity_merged.head() # check the last columns!
#=============================================#
##=======================================================================##

,Postal Code,City,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,35801,Huntsville,Madison,34.7298,-86.5859,0,American Restaurant,Park,Arcade,Beer Store,Coffee Shop,Gym / Fitness Center,Sushi Restaurant,Pizza Place,Speakeasy,Outdoors & Recreation
1,35806,Huntsville,Madison,34.7628,-86.684,1,Hotel,Sports Bar,Salon / Barbershop,Pizza Place,BBQ Joint,Outdoors & Recreation,Steakhouse,Bakery,Grocery Store,Beer Store
2,35807,Huntsville,Madison,34.6686,-86.4418,1,Sushi Restaurant,Gym / Fitness Center,Arcade,BBQ Joint,Bakery,Beer Store,Coffee Shop,Grocery Store,Hotel,Steakhouse


In [16]:
#%%===== Visualize the result (Main target city) =====%%#
## Currently editing
# create map
map_clusters = folium.Map(location=[metro_latitude, metro_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, postal, cluster in zip(TargetCity_merged['Latitude'], TargetCity_merged['Longitude'], TargetCity_merged['City'], TargetCity_merged['Postal Code'], TargetCity_merged['Cluster Labels']):
    label = folium.Popup(str(postal) + ' ' + str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters
# print('Saving the map...')
# map_clusters_save_name = map_save_path + 'Clustering_' + search_city + '.html'
# map_clusters.save(map_clusters_save_name)
# print('{} clustering map saved successfully.'.format(search_city))
##==============================================================##

In [19]:
#%%===== Examine the clustering results of the target city (Birmingham) =====%%#
# Cluster 1
TargetCity_merged.loc[TargetCity_merged['Cluster Labels'] == 0, TargetCity_merged.columns[[1] + list(range(5, TargetCity_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Birmingham,0,Wine Bar,Hotel,Antique Shop,Asian Restaurant,Bar,Coffee Shop,Gym,American Restaurant,Mexican Restaurant,Southern / Soul Food Restaurant


In [20]:
# Cluster 2
TargetCity_merged.loc[TargetCity_merged['Cluster Labels'] == 1, TargetCity_merged.columns[[1] + list(range(5, TargetCity_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Birmingham,1,Health Food Store,Pet Store,Cajun / Creole Restaurant,Cupcake Shop,Furniture / Home Store,Tea Room,American Restaurant,Pizza Place,Sandwich Place,Supermarket


In [21]:
#%%===== Visualizae the result (compare city) =====%%#
## Currently editing
# create map
map_clusters = folium.Map(location=[compare_latitude, compare_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, postal, cluster in zip(compareCity_merged['Latitude'], compareCity_merged['Longitude'], compareCity_merged['City'], compareCity_merged['Postal Code'], compareCity_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' ' + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters
# print('Saving the map...')
# map_clusters_save_name = map_save_path + 'Clustering_' + compare_city + '.html'
# map_clusters.save(map_clusters_save_name)
# print('{} clustering map saved successfully.'.format(compare_city))
##==============================================================##

In [23]:
#%%===== Examine the clustering results of the compare city (Huntsville) =====%%#
# Cluster 1
compareCity_merged.loc[compareCity_merged['Cluster Labels'] == 0, compareCity_merged.columns[[1] + list(range(5, compareCity_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Huntsville,0,American Restaurant,Park,Arcade,Beer Store,Coffee Shop,Gym / Fitness Center,Sushi Restaurant,Pizza Place,Speakeasy,Outdoors & Recreation


In [24]:
# Cluster 2
compareCity_merged.loc[compareCity_merged['Cluster Labels'] == 1, compareCity_merged.columns[[1] + list(range(5, compareCity_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Huntsville,1,Hotel,Sports Bar,Salon / Barbershop,Pizza Place,BBQ Joint,Outdoors & Recreation,Steakhouse,Bakery,Grocery Store,Beer Store
2,Huntsville,1,Sushi Restaurant,Gym / Fitness Center,Arcade,BBQ Joint,Bakery,Beer Store,Coffee Shop,Grocery Store,Hotel,Steakhouse
